In [42]:
#modules needed for using alpaca and formatting our data
import alpaca_trade_api as tradeapi
import requests  
import json
import pandas as pd
import tensorflow as tf
import numpy as np

In [43]:
api = tradeapi.REST(
    key_id='PKFH5ZA4MRGSTSEFNQ3S',
    secret_key='Yy5ygxRCE6bOwRRC03tQ9mYMudA6WQRKkRMbTnFY',
    base_url='https://paper-api.alpaca.markets'
)

In [44]:
def getData(symbol, timeframe, limit):
    barset = api.get_barset(symbol, timeframe, limit)
    return barset


DATA = getData("ADBE", "day", 5)

In [45]:
#change data pulled into same format as data from quandl so that we may use our trained Model on it
def changeFormat(data):
    o = []
    h = []
    l = []
    c = []
    v = []
    for d in data:
        for i in data[d]:
            o.append(i.o)
            h.append(i.h)
            l.append(i.l)
            c.append(i.c)
            v.append(i.v)
    dataDict = {"open":o, "high":h, "low":l, "close":l, "volume":v}
    df = pd.DataFrame(dataDict)
    return df


formatted_data = changeFormat(DATA)
print(formatted_data)

full_data = len(formatted_data)

     open      high      low    close   volume
0  449.84  450.7400  442.280  442.280  1561978
1  448.26  449.0799  440.700  440.700  2346534
2  449.09  454.0350  446.345  446.345  2065372
3  451.35  464.3700  449.130  449.130  2066577
4  460.87  461.7900  445.090  445.090  3049001


In [46]:
#Testing Model 1 on Alpaca data
dataset = formatted_data.values
data_mean = dataset[:full_data].mean(axis = 0)
data_std = dataset[:full_data].std(axis = 0)

dataset = (dataset-data_mean)/data_std
dataset

array([[-0.44317835, -0.86994955, -0.81567936, -0.81567936, -1.34808214],
       [-0.78608813, -1.14435744, -1.34625712, -1.34625712,  0.26439341],
       [-0.60595198, -0.32529922,  0.54938305,  0.54938305, -0.31347082],
       [-0.11546076,  1.38303495,  1.48461031,  1.48461031, -0.31099422],
       [ 1.95067922,  0.95657126,  0.12794312,  0.12794312,  1.70815377]])

In [47]:
#do multivariate data function on this to further organize for Model 1
##had difficulty importing the below functions from Model_test.ipynb file
def multivariate_data(dataset, target, start_index, end_index, history_size,
                     target_size, step, single_step = False):
    data = []
    labels = []
    
    start_index = start_index+history_size
    if end_index is None:
        end_index = len(dataset) - target_size
        
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        
        if single_step:
            labels.append(dataset[indices])
        else:
            labels.append(target[i:i+target_size])
            
    return np.array(data), np.array(labels)
            

In [48]:
past_history = 1
future_target = 1
STEP = 1

In [49]:
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               0, full_data, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [50]:
BATCH_SIZE = 5
BUFFER_SIZE = full_data

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [51]:
#imports our saved model into here

new_model_1 = tf.keras.models.load_model('saved_model_1/original_df_model')

In [52]:
#using processed data 'val_data_single' instead of normalized, raw 'dataset'
predictions = new_model_1.predict(val_data_single, steps=1)
predictions

array([[-0.57437277],
       [-0.5610862 ],
       [-0.21160915],
       [-0.05105156]], dtype=float32)

In [53]:
#for removing our normalization pre-processing so that we may compare our predicted values to those from the original dataset
def price(predictions):
    prices = []
    for prediction in predictions:
        price = prediction*data_std+data_mean
        prices.append(price)
    return prices

In [54]:
#compare these predictions to each column for days 2-5 (rows 1-4) in our original pull! not bad, huh :)
price(predictions)

[array([4.49235505e+02, 4.52528167e+02, 4.42998583e+02, 4.42998583e+02,
        1.93842918e+06]),
 array([4.49296725e+02, 4.52608547e+02, 4.43038149e+02, 4.43038149e+02,
        1.94489382e+06]),
 array([4.50906984e+02, 4.54722797e+02, 4.44078852e+02, 4.44078852e+02,
        2.11493318e+06]),
 array([4.51646773e+02, 4.55694131e+02, 4.44556974e+02, 4.44556974e+02,
        2.19305307e+06])]